___
<img style="float: right; margin: 0px 0px 15px 15px;" src="https://www.carrerasenlinea.mx/logos/original/logo-instituto-tecnologico-y-de-estudios-superiores-de-occidente.png" width="150px" height="100px" />

# Microestructuras y Sistemas de Trading
## Laboratorio 3
### Equipo C

## <font color="navy">  "PARADOX OF CHOICE"
    
* <font color="black">**INTEGRANTE 1 :**</font> Mariana López Correa
* <font color="black">**INTEGRANTE 2 :**</font> Axel Francisco Munguía Quintero
* <font color="black">**INTEGRANTE 2 :**</font> Marysol Cantarero Velazquez Del Valle 
* <font color="black">**INTEGRANTE 2 :**</font> Gaspar Benitez Fregoso

### <font color = navy> Situación
Utilizar la información histórica de la cuenta de trader manual y, mediante la formulación y programación de 1 "Sesgo cognitivo", realizar una evaluación del desempeño de la cuenta utilizando el marco de "Behavioral economics". 

### <font color = navy> Objetivo
Inferir si al momento de tomar las deciciones que se tomaron encuenta, se estuvo incurriendo en el sesgo cognitivo seleccionado y su impacto en el resultado final.
    
### <font color = navy> Sesgo cognitivo que elegimos    
Paradox of Choice

El sesgo cognitivo Paradox of Choice o paradoja de elección esta basado en un libro de Barry Shchartz(Paradox of choice) que explica que, si bien la sabiduría convencional nos dice que más opciones conducen a más libertad y más felicidad, la investigación nos muestra que dar a las personas muchas opciones no aumenta el rendimiento o la satisfacción, en incluso a menudo conduce a la parálisis y la instatisfacción.

Ejemplos:

1). Cincuente estudios independientes encontraron una conexión significativa entre la elección y la ansiedad.

2). Un estudio de Vanguard mostró que por cada diez fondos mutuos agregados a un plan de jubilación, la participación se redujo en un 2% adicional (Schartz, 2005).

3). Todos nos hemos sentido abrumados al estar frente a un ligar lleno de diferentes formas del mismo productos, y sentirnos incapacitados para tomar una decisión. (Cuando tienes 20 opciones de helados será mas díficil escoger a si tienes solo vainilla, fresa y chocolate).

En este proyecto se realiza el análisis de este sesgo cognitivo para este inversionista para buscar si existe una relación del mismo cuando las posibilidades de elección de inversión incrementan. 

Para esto se separaron las transacciones por día, y por activos donde se obtuvieron valores con rendimientos positivos y negativos. Esto con el objetivo de conocer el número de activos que obtuvieron rendimiento positivo en el día. Después se realizo también el análisis pero de forma semanal y mensual.

Al final se muestra la información general sobre el comportamiento en las ganancias y pérdidas del inversionista en los distintios plazos, así como los activos que explican más del 95% de los ingresos del inversionista y en los que posiblemente debería enfocarse más.

### <font color = navy> Procedimiento

Importando librerías

In [1]:
import numpy as np
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objs as go # Si no se puede con plotly.graph_objects trate plotly.graph_objs
from plotly.offline import init_notebook_mode, plot
from datetime import datetime , timedelta
init_notebook_mode(connected=True)
import json
#%% Opciones para visualizar data frames en consola
pd.set_option("display.max_rows",5000)
pd.set_option("display.max_columns",500)
pd.set_option("display.width",1000)
#style.use("ggplot")

In [2]:
# Ingreso de datos de entrada: leer archivo JSON
archivo = "archivo_tradeview_2.xlsx"
def f_datosent(P0_archivo="archivo_tradeview_0.json"):
    """
    param P0_archivio: Nombre del archivo con ubicación en la misma carpeta.​
    """
    P1_tipo = P0_archivo.split(".")
    P1_tipo = P1_tipo[1]
    if P1_tipo=="json":
        with open(P0_archivo, encoding="utf-8-sig") as json_file:
            datos_2 = json.load(json_file)["data"]       
        df_datos = pd.DataFrame(datos_2["closedTransactions"]["list"])
        # Resetear index para tener todos los números de índices completos
        df_datos = df_datos.drop("info",axis=1).dropna().reset_index().drop("index",axis=1)
        # Tomar deseados
        df_datos = df_datos[(df_datos["type"]=="buy") | (df_datos["type"]=="sell") | (df_datos["type"]=="s/l") | (df_datos["type"]=="t/p")]
        # Renombrar
        df_datos.rename(columns  = {"SL":"sl","TP":"tp","price":"openPrice","price2":"closePrice","item":"instrument"},inplace=True) # Con inplace se guarda 
        return df_datos
    elif P1_tipo=="xlsx":   
        df_datos = pd.read_excel(archivo)
        return df_datos
    elif P1_tipo=="csv":
        df_datos = pd.read_excel(archivo)
        return df_datos
    else:
        print("No manejamos tu tipo de extensión")
df_datos = f_datosent(archivo)
df_datos.head(3) # Mostrar 3 primeros

,Order,openTime,Type,Size,Symbol,openPrice,S/L,T/P,closeTime,closePrice,Commission,Taxes,Swap,Profit
0,24316034,2019.08.27 06:57:50,buy,0.04,xauusd,1529.210,1523.00,1543.0,2019.08.27 09:16:01,1533.690,0.0,0,0,17.92
1,24334215,2019.08.27 17:06:20,buy,0.01,eurjpy,117.664,116.95,118.7,2019.08.29 06:38:16,117.374,0.0,0,0,-2.73
2,24342155,2019.08.27 19:25:39,buy,0.05,eurjpy,117.346,116.95,119.7,2019.08.28 21:01:24,117.452,0.0,0,0,5.00


Descargando información de trader 

In [3]:
data= pd.read_excel("archivo_tradeview_2.xlsx",index_col='openTime').sort_index()
#data.index=pd.DatetimeIndex(data.index)
data.head() #primeros 5 datos

,Order,Type,Size,Symbol,openPrice,S/L,T/P,closeTime,closePrice,Commission,Taxes,Swap,Profit
openTime,,,,,,,,,,,,,
2019.08.27 06:18:03,24311675,sell,0.04,eurjpy,117.40700,117.7000,0.000,2019.08.27 06:20:11,117.42300,0.0,0,0,-0.61
2019.08.27 06:22:44,24311730,sell,0.04,eurjpy,117.38900,117.7000,0.000,2019.08.27 06:23:28,117.41200,0.0,0,0,-0.87
2019.08.27 06:23:44,24311745,sell,0.04,eurjpy,117.36800,0.0000,0.000,2019.08.27 06:24:13,117.40200,0.0,0,0,-1.29
2019.08.27 06:45:16,24315841,sell,0.05,eurjpy,117.34300,117.6500,117.100,2019.08.27 09:16:08,117.31500,0.0,0,0,1.32
2019.08.27 06:52:01,24315970,sell,0.04,eurusd,1.11005,1.1126,1.107,2019.08.27 09:16:04,1.11027,0.0,0,0,-0.88


In [4]:
data= pd.read_excel("archivo_tradeview_2.xlsx")
data.index = data["openTime"]; data = data.drop("openTime",axis=1); 
data = data.sort_index() 

In [5]:
data.tail() #últimos 5 datos

,Order,Type,Size,Symbol,openPrice,S/L,T/P,closeTime,closePrice,Commission,Taxes,Swap,Profit
openTime,,,,,,,,,,,,,
2019.09.25 01:54:46,25159303,sell,0.01,xauusd,1531.28000,1537.0000,0.000,2019.09.25 15:24:35,1527.36000,0.00,0,0,3.92
2019.09.25 18:22:33,25188529,buy,0.08,usdmxn,19.56233,19.5900,19.900,2019.09.26 21:00:44,19.57897,0.00,0,0,6.80
2019.09.25 18:23:20,25188553,buy,0.01,usdmxn,19.56103,19.5900,19.900,2019.09.26 21:00:44,19.58247,0.00,0,0,1.09
2019.09.25 20:05:46,25193432,buy,0.08,eurusd,1.09455,1.0915,1.111,2019.09.26 22:33:46,1.09148,0.00,0,0,-24.56
2019.09.26 20:20:58,25229583,buy,0.09,btcusd,7910.75000,0.0000,0.000,2019.09.27 15:49:19,7986.78000,-1.07,0,0,6.84


In [6]:
data.index = [data.index[k][:10] for k in range(data.shape[0])]  # Quitando la hora del índice, dejando solo fecha : "yyy.mm.dd"
data.head() #primeros 5 datos

,Order,Type,Size,Symbol,openPrice,S/L,T/P,closeTime,closePrice,Commission,Taxes,Swap,Profit
2019.08.27,24311675,sell,0.04,eurjpy,117.40700,117.7000,0.000,2019.08.27 06:20:11,117.42300,0.0,0,0,-0.61
2019.08.27,24311730,sell,0.04,eurjpy,117.38900,117.7000,0.000,2019.08.27 06:23:28,117.41200,0.0,0,0,-0.87
2019.08.27,24311745,sell,0.04,eurjpy,117.36800,0.0000,0.000,2019.08.27 06:24:13,117.40200,0.0,0,0,-1.29
2019.08.27,24315841,sell,0.05,eurjpy,117.34300,117.6500,117.100,2019.08.27 09:16:08,117.31500,0.0,0,0,1.32
2019.08.27,24315970,sell,0.04,eurusd,1.11005,1.1126,1.107,2019.08.27 09:16:04,1.11027,0.0,0,0,-0.88


In [7]:
data.tail() #últimos 5 datos

,Order,Type,Size,Symbol,openPrice,S/L,T/P,closeTime,closePrice,Commission,Taxes,Swap,Profit
2019.09.25,25159303,sell,0.01,xauusd,1531.28000,1537.0000,0.000,2019.09.25 15:24:35,1527.36000,0.00,0,0,3.92
2019.09.25,25188529,buy,0.08,usdmxn,19.56233,19.5900,19.900,2019.09.26 21:00:44,19.57897,0.00,0,0,6.80
2019.09.25,25188553,buy,0.01,usdmxn,19.56103,19.5900,19.900,2019.09.26 21:00:44,19.58247,0.00,0,0,1.09
2019.09.25,25193432,buy,0.08,eurusd,1.09455,1.0915,1.111,2019.09.26 22:33:46,1.09148,0.00,0,0,-24.56
2019.09.26,25229583,buy,0.09,btcusd,7910.75000,0.0000,0.000,2019.09.27 15:49:19,7986.78000,-1.07,0,0,6.84


Separando la información por fecha.

In [8]:
dataframes = []  # lista vacia
unicos = np.unique(data.index)  # del índice se buscan las fechas únicas para clasificar la información de acuerdo a estas.
for k in range(len(unicos)):
    dataframes.append(pd.DataFrame(data.loc[data.index==unicos[k],:])) #agregando a la lista los dataframes con la info separada.

In [9]:
dataframes[0] # información del día 27/08/2019

,Order,Type,Size,Symbol,openPrice,S/L,T/P,closeTime,closePrice,Commission,Taxes,Swap,Profit
2019.08.27,24311675,sell,0.04,eurjpy,117.40700,117.7000,0.000,2019.08.27 06:20:11,117.42300,0.0,0,0,-0.61
2019.08.27,24311730,sell,0.04,eurjpy,117.38900,117.7000,0.000,2019.08.27 06:23:28,117.41200,0.0,0,0,-0.87
2019.08.27,24311745,sell,0.04,eurjpy,117.36800,0.0000,0.000,2019.08.27 06:24:13,117.40200,0.0,0,0,-1.29
2019.08.27,24315841,sell,0.05,eurjpy,117.34300,117.6500,117.100,2019.08.27 09:16:08,117.31500,0.0,0,0,1.32
2019.08.27,24315970,sell,0.04,eurusd,1.11005,1.1126,1.107,2019.08.27 09:16:04,1.11027,0.0,0,0,-0.88
2019.08.27,24316034,buy,0.04,xauusd,1529.21000,1523.0000,1543.000,2019.08.27 09:16:01,1533.69000,0.0,0,0,17.92
2019.08.27,24318553,sell,0.05,eurgbp,0.90821,0.9120,0.903,2019.08.27 13:31:11,0.90559,0.0,0,0,16.07
2019.08.27,24325707,sell,0.04,eurusd,1.11060,1.1130,1.107,2019.08.27 13:57:03,1.11093,0.0,0,0,-1.32
2019.08.27,24325723,sell,0.04,eurusd,1.11062,1.1130,1.107,2019.08.27 13:57:57,1.11079,0.0,0,0,-0.68
2019.08.27,24325740,sell,0.05,eurusd,1.11049,1.1130,1.107,2019.08.27 14:02:05,1.11090,0.0,0,0,-2.05


In [10]:
dataframes[1] # información del día 28/08/2019

,Order,Type,Size,Symbol,openPrice,S/L,T/P,closeTime,closePrice,Commission,Taxes,Swap,Profit
2019.08.28,24392456,buy,0.04,eurusd,1.10791,1.1015,1.1175,2019.08.30 17:57:15,1.10150,0.0,0,0,-25.64
2019.08.28,24373260,buy,0.01,eurusd,1.10791,1.1040,1.1175,2019.08.29 06:38:28,1.10818,0.0,0,0,0.27


In [11]:
dataframes[2] # información del día 29/08/2019

,Order,Type,Size,Symbol,openPrice,S/L,T/P,closeTime,closePrice,Commission,Taxes,Swap,Profit
2019.08.29,24392440,buy,0.04,gbpusd,1.22024,1.2080,1.237,2019.08.29 15:01:43,1.22173,0.0,0,0,5.96
2019.08.29,24392415,buy,0.01,gbpusd,1.22024,1.2080,1.237,2019.08.29 06:37:23,1.21990,0.0,0,0,-0.34
2019.08.29,24392425,buy,0.01,gbpjpy,129.23400,128.0000,132.200,2019.08.29 06:36:58,129.21000,0.0,0,0,-0.23
2019.08.29,24392432,buy,0.04,gbpjpy,129.23400,128.0000,132.200,2019.08.29 15:01:24,129.80100,0.0,0,0,21.35
2019.08.29,24416172,buy,0.04,eurusd,1.10543,1.1040,0.000,2019.08.29 19:56:51,1.10518,0.0,0,0,-1.00
2019.08.29,24416187,buy,0.05,eurusd,1.10545,1.1015,0.000,2019.08.30 17:57:14,1.10150,0.0,0,0,-19.75
2019.08.29,24417804,buy,0.05,xauusd,1528.76000,1524.0000,1555.000,2019.08.29 21:11:08,1523.80000,0.0,0,0,-24.80


##### Función para seleccionar índice y valores con rendimientos "positivos"

Con estas función creada, el objetivo es conocer el número de activos que tuvieron rendimiento positivo durante el día ( Si se tuvieron más posiciones de pérdida que de ganancia sobre un activo no se tomará)

(a)En la primera parte se hace dataframe con sumas por símbolo, (b)en la segunda parte se hace lista del dataframe con valores True si es positivo y False negativo. (c)Juntando estas dos partes del código se crea un dataframe con los valores cuando sea True y NaN cuando sea False, de tal manera que .dropna() eliminará los valores negativos. La última parte selecciona el índice de estos valores y los mismos valores en formato de lista.


In [12]:
def choose_index_values(dataframe,l):
    a = dataframes[l].iloc[:,[3,12]].groupby("Symbol").sum()
    #b = [dataframes[l].iloc[:,[3,12]].groupby("Symbol").sum()>0]
    c = a[a>0].dropna()
    index = c.index.values.tolist()
    values = c.values.tolist()
    return index, values

In [13]:
choose_index_values(dataframes,0)[0]

['eurgbp', 'xauusd']

In [14]:
profits_day = [choose_index_values(dataframes,l) for l in range(len(dataframes)) if len(choose_index_values(dataframes,l)[0]) ]
profits_day

[(['eurgbp', 'xauusd'], [[16.07], [17.92]]),
 (['gbpjpy', 'gbpusd'], [[21.12], [5.62]]),
 (['usdjpy', 'xauusd'], [[20.41], [17.8]]),
 (['audusd'], [[10.270000000000001]]),
 (['usdcad'], [[29.11]]),
 (['eurusd', 'gbpusd'], [[3.0], [9.35]]),
 (['usdcad'], [[11.52]]),
 (['eurusd', 'usdjpy', 'xauusd'],
  [[22.240000000000002], [2.14], [19.860000000000003]]),
 (['gbpusd'], [[21.78]]),
 (['eurusd', 'usdmxn', 'xauusd'], [[17.22], [43.0], [93.06]]),
 (['eurusd', 'gbpusd', 'xauusd'], [[23.34], [34.32], [3.42]]),
 (['eurusd'], [[10.979999999999999]]),
 (['eurusd', 'gbpusd', 'usdmxn'], [[30.42], [1.56], [15.33]]),
 (['xauusd'], [[50.29]]),
 (['usdmxn', 'xauusd'], [[7.89], [25.880000000000003]]),
 (['btcusd'], [[6.84]])]

In [15]:
[len(profits_day[l][0]) for l in range(len(profits_day))]

[2, 2, 2, 1, 1, 2, 1, 3, 1, 3, 3, 1, 3, 1, 2, 1]

Graficando ganancias para analizar la información

In [16]:
# Inicializando figura con subplots 
fig = make_subplots(
    rows=2, cols=2, subplot_titles=("No. Activos diarios que obtuvieron rendimiento", 
                                    "Frecuencia de Activos Utilizados cuando son positivos", 
                                    "Ganancia Promedio por día", "Frecuecia Ganancia Promedio por día")
)

# Agregando figuras(trazos)
fig.add_trace(go.Scatter( y=[len(profits_day[l][0]) for l in range(len(profits_day))if len(profits_day[l][0])>0]), row=1, col=1)
fig.add_trace(go.Histogram( x=[len(profits_day[l][0]) for l in range(len(profits_day))if len(profits_day[l][0])>0],
        xbins=dict(size=0.3)), row=1, col=2)
fig.add_trace(go.Scatter( y=[np.mean(profits_day[l][1]) for l in range(len(profits_day)) if len(profits_day[l][1])>0]), row=2, col=1)
fig.add_trace(go.Histogram(x=[np.mean(profits_day[l][1]) for l in range(len(profits_day)) if len(profits_day[l][1])>0],
        xbins=dict(size=4)), row=2, col=2)

# Actualzando los ejes "x"
fig.update_xaxes(title_text="Días", row=1, col=1)
fig.update_xaxes(title_text="No. Activos",row=1, col=2)
fig.update_xaxes(title_text="Días", row=2, col=1)
fig.update_xaxes(title_text="Dinero", row=2, col=2)

# Actualzando los ejes "y"
fig.update_yaxes(title_text="No. Activos", row=1, col=1)
fig.update_yaxes(title_text="Frecuencia", row=1, col=2)
fig.update_yaxes(title_text="Dinero",row=2, col=1)
fig.update_yaxes(title_text="Frecuencia", row=2, col=2)

# Actualizando título
fig.layout.update(showlegend=False, title_text="Análisis de Ganancias", height=700)

#plot(fig,filename= 'basic-scatter')
fig.show()

##### Función para seleccionar índice y valores con rendimientos "negativos"

Con estas función creada, el objetivo es conocer el número de activos que tuvieron rendimiento positivo durante el día ( Si se tuvieron más posiciones de pérdida que de ganancia sobre un activo no se tomará). En la función final se hará un análisis de pérdidas y ganancias diarias, semanales y mensuales.

**a**) En la primera parte se hace dataframe con sumas por símbolo.

**b**) En la segunda parte se hace lista del dataframe con valores True si es negativo y False positivo. 

**c**) Juntando estas dos partes del código se crea un dataframe con los valores cuando sea True y NaN cuando sea False, de tal manera que .dropna() eliminará los valores positivos. La última parte selecciona el índice de estos valores y los mismos valores en formato de lista.

In [17]:
def choose_index_values_2(dataframe,l):
    a = dataframes[l].iloc[:,[3,12]].groupby("Symbol").sum()
    #b = [dataframes[l].iloc[:,[3,12]].groupby("Symbol").sum()<0]
    c = a[a<0].dropna()
    index = c.index.values.tolist()
    values = c.values.tolist()
    return index, values

In [18]:
no_profits_day = [choose_index_values_2(dataframes,l) for l in range(len(dataframes)) if len(choose_index_values_2(dataframes,l)[0])>=1]
no_profits_day

[(['eurjpy', 'eurusd'], [[-9.16], [-4.93]]),
 (['eurusd'], [[-25.37]]),
 (['eurusd', 'xauusd'], [[-20.75], [-24.8]]),
 (['eurusd', 'gbpusd', 'usdcad'], [[-25.8], [-33.64], [-18.200000000000003]]),
 (['gbpjpy', 'usdjpy', 'xauusd'], [[-14.55], [-21.88], [-68.0]]),
 (['audusd'], [[-12.1]]),
 (['eurusd'], [[-4.5]]),
 (['usdmxn'], [[-29.39]]),
 (['eurusd'], [[-24.56]])]

Graficando pérdidas para analizar la información

In [19]:
# Inicializando figura con subplots 
fig = make_subplots(
    rows=2, cols=2, subplot_titles=("No. Activos diarios que no obtuvieron rendimiento", 
                                    "Frecuencia de Activos Utilizados cuando no son positivos", 
                                    "Pérdida Promedio por día", "Frecuecia Pérdida Promedio por día")
)

# Agregando figuras(trazos)
fig.add_trace(go.Scatter( y=[len(no_profits_day[l][0]) for l in range(len(no_profits_day))if len(no_profits_day[l][0])>0]), row=1, col=1)
fig.add_trace(go.Histogram( x=[len(no_profits_day[l][0]) for l in range(len(no_profits_day))if len(no_profits_day[l][0])>0],
        xbins=dict(size=0.3)), row=1, col=2)
fig.add_trace(go.Scatter( y=[np.mean(no_profits_day[l][1]) for l in range(len(no_profits_day)) if len(no_profits_day[l][1])>0]), row=2, col=1)
fig.add_trace(go.Histogram(x=[np.mean(no_profits_day[l][1]) for l in range(len(no_profits_day)) if len(no_profits_day[l][1])>0],
        xbins=dict(size=4)), row=2, col=2)

# Actualzando los ejes "x"
fig.update_xaxes(title_text="Días", row=1, col=1)
fig.update_xaxes(title_text="No. Activos",row=1, col=2)
fig.update_xaxes(title_text="Días", row=2, col=1)
fig.update_xaxes(title_text="Dinero", row=2, col=2)

# Actualzando los ejes "y"
fig.update_yaxes(title_text="No. Activos", row=1, col=1)
fig.update_yaxes(title_text="Frecuencia", row=1, col=2)
fig.update_yaxes(title_text="Dinero",row=2, col=1)
fig.update_yaxes(title_text="Frecuencia", row=2, col=2)

# Actualizando título
fig.layout.update(showlegend=False, title_text="Análisis de Pérdidas", height=700)

#plot(fig,filename= 'basic-scatter')
fig.show()

Función para gráficar automáticamente análisis de pérdidas o ganancias

In [20]:
#FUNCIÓN PARA GRÁFICAR AUTOMÁTICAMENTE PÉRDIDAS O GANANCIAS
def subplots_positivos_negativos(profits_day,positivo =True, day =True):
    if positivo:
        utilizar = ["rendimiento", "son positivos", "Ganancia"]
    else:
        utilizar = ["pérdida", "no son positivos", "Pérdida"]
    
    if day:
        periodo = ["Días"]
    else:
        periodo = ["Semanas"]
        
    fig = make_subplots(
        rows=2, cols=2, subplot_titles=("No. Activos diarios que obtuvieron "+ utilizar[0],
                                        "Frecuencia de Activos Utilizados cuando " + utilizar[1],
                                        utilizar[2]+ " Promedio por día", "Frecuecia " +utilizar[2]+ " Promedio por día")
    )

    # Add traces
    fig.add_trace(go.Scatter( y=[len(profits_day[l][0]) for l in range(len(profits_day))]), row=1, col=1)
    fig.add_trace(go.Histogram( x=[len(profits_day[l][0]) for l in range(len(profits_day))],
            xbins=dict(size=0.3)), row=1, col=2)
    fig.add_trace(go.Scatter( y=[np.mean(profits_day[l][1]) for l in range(len(profits_day)) if len(profits_day[l][1])>0]), row=2, col=1)
    fig.add_trace(go.Histogram(x=[np.mean(profits_day[l][1]) for l in range(len(profits_day)) if len(profits_day[l][1])>0],
            xbins=dict(size=4.5)), row=2, col=2)

    # Update xaxis properties
    fig.update_xaxes(title_text=periodo[0], row=1, col=1)
    fig.update_xaxes(title_text="No. Activos",row=1, col=2)
    fig.update_xaxes(title_text=periodo[0], row=2, col=1)
    fig.update_xaxes(title_text="Dinero", row=2, col=2)

    # Update yaxis properties
    fig.update_yaxes(title_text="No. Activos", row=1, col=1)
    fig.update_yaxes(title_text="Frecuencia", row=1, col=2)
    fig.update_yaxes(title_text="Dinero",row=2, col=1)
    fig.update_yaxes(title_text="Frecuencia", row=2, col=2)

    # Update title and height
    fig.layout.update(showlegend=False, title_text="Análisis de " + utilizar[2]+"s", height=700)
    #plot(fig,filename= 'basic-scatter')
    fig.show()
    return 
#subplots_positivos_negativos(profits_day=profits_day,positivo=True) # gráfica de análisis de ganancias
#subplots_positivos_negativos(profits_day=profits_day,positivo=False) # gráfica de análisis de pérdidas

Función que realiza todo el código del sesgo cognitivo "Paradox of choice"

In [21]:
def f_bf_Paradox_Choice(dataframe):
    # Importar librerías
    import numpy as np
    import pandas as pd
    from plotly.subplots import make_subplots
    import plotly.graph_objs as go # Si no se puede con plotly.graph_objects trate plotly.graph_objs
    from plotly.offline import plot
    from datetime import datetime , timedelta
    init_notebook_mode(connected=True)
    #style.use("ggplot")
    
    # Definir funciones
    
    def choose_index_values(dataframe,l):
        a = dataframes[l].iloc[:,[3,12]].groupby("Symbol").sum()
        #b = [dataframes[l].iloc[:,[3,12]].groupby("Symbol").sum()>0]
        c = a[a>0].dropna()
        index = c.index.values.tolist()
        values = c.values.tolist()
        return index, values
    
    def choose_index_values_2(dataframe,l):
        a = dataframes[l].iloc[:,[3,12]].groupby("Symbol").sum()
        #b = [dataframes[l].iloc[:,[3,12]].groupby("Symbol").sum()<0]
        c = a[a<0].dropna()
        index = c.index.values.tolist()
        values = c.values.tolist()
        return index, values

    
    def subplots_positivos_negativos(profit_loss,positivo =True, day =True):
        # Para saber si son pérdidas o ganancias y si es diario o semanal
        if positivo:
            utilizar = ["rendimiento", "son positivos", "Ganancia"]
        else:
            utilizar = ["pérdida", "no son positivos", "Pérdida"]

        if day:
            periodo = ["Días"]
        else:
            periodo = ["Semanas"]

        fig = make_subplots(
            rows=2, cols=2, subplot_titles=("No. Activos diarios que obtuvieron "+ utilizar[0],
                                            "Frecuencia de Activos Utilizados cuando " + utilizar[1],
                                            utilizar[2]+ " Promedio por "+str(periodo), "Frecuecia " +utilizar[2]+ " Promedio por "+str(periodo))
        )

        # Add traces
        fig.add_trace(go.Scatter( y=[len(profit_loss[l][0]) for l in range(len(profit_loss))]), row=1, col=1)
        fig.add_trace(go.Histogram( x=[len(profit_loss[l][0]) for l in range(len(profit_loss))],
                xbins=dict(size=0.3)), row=1, col=2)
        fig.add_trace(go.Scatter( y=[np.mean(profit_loss[l][1]) for l in range(len(profit_loss)) if len(profit_loss[l][1])>0]), row=2, col=1)
        fig.add_trace(go.Histogram(x=[np.mean(profit_loss[l][1]) for l in range(len(profit_loss)) if len(profit_loss[l][1])>0],
                xbins=dict(size=4.5)), row=2, col=2)

        # Update xaxis properties
        fig.update_xaxes(title_text=periodo[0], row=1, col=1)
        fig.update_xaxes(title_text="No. Activos",row=1, col=2)
        fig.update_xaxes(title_text=periodo[0], row=2, col=1)
        fig.update_xaxes(title_text="Dinero", row=2, col=2)

        # Update yaxis properties
        fig.update_yaxes(title_text="No. Activos", row=1, col=1)
        fig.update_yaxes(title_text="Frecuencia", row=1, col=2)
        fig.update_yaxes(title_text="Dinero",row=2, col=1)
        fig.update_yaxes(title_text="Frecuencia", row=2, col=2)

        # Update title and height
        fig.layout.update(showlegend=False, title_text="Análisis de " + utilizar[2]+"s", height=700)

        #plot(fig,filename= 'basic-scatter')
       
        return

    # Ajustar en Index el openTime y borrarlo como columna
    data = dataframe.copy() # Este dataset se utilizará cuando se realice el análisis semanal.
    num_Activos = len(np.unique(data["Symbol"].values))
    dataframe.index = dataframe["openTime"]; dataframe = dataframe.drop("openTime",axis=1); 
    dataframe = dataframe.sort_index() 
    data_mensual = dataframe.copy()
    dataframe.index = [dataframe.index[k][:10] for k in range(dataframe.shape[0])]  # Quitando la hora del índice, dejando solo fecha : "yyy.mm.dd"
    dataframes = []  # lista vacia
    unicos = np.unique(dataframe.index)  # del índice se buscan las fechas únicas para clasificar la información de acuerdo a estas.
    
    for k in range(len(unicos)):
        dataframes.append(pd.DataFrame(dataframe.loc[dataframe.index==unicos[k],:])) #agregando a la lista los dataframes con la info separada.  
    
    # Conocer los potiviso y negativos diarios
    profits_day = [choose_index_values(dataframes,l) for l in range(len(dataframes)) if len(choose_index_values(dataframes,l)[0]) ]
    no_profits_day = [choose_index_values_2(dataframes,l) for l in range(len(dataframes)) if len(choose_index_values_2(dataframes,l)[0])>=1]
    # Para rellenar tabla
    num_Activos_ganancia_day = len(np.unique(np.concatenate([l[0] for l in profits_day])))
    num_Activos_perdida_day = len(np.unique(np.concatenate([l[0] for l in no_profits_day])))
    # Graficar
    diario_positivo = subplots_positivos_negativos(profit_loss=profits_day,positivo =True, day =True)
    diario_negativo = subplots_positivos_negativos(profit_loss=no_profits_day,positivo =False, day =True)
    
    # Análisis semanal
    data2= data
    data2["'openTime'"] = [data2['openTime'][k][:10] for k in range(data2.shape[0])]
    data2['openTime'] = [datetime.strptime(data2["'openTime'"][k], '%Y.%m.%d') for k in range(data2.shape[0])]
    dty=[]
    gr = data2.groupby(pd.Grouper(key='openTime',freq='W'))
    for name, group in gr:
        if len(group) > 0:        
            dty.append(group)   
    dty=[pd.DataFrame(dty[i]).set_index('openTime') for i in range(len(dty))]
    dty =[l.drop("'openTime'",axis=1) for l in  dty]
    dataframes=dty
    # Conocer los positivos y Negativos semanles
    profits_week = [choose_index_values(dataframes,l) for l in range(len(dataframes)) if len(choose_index_values(dataframes,l)[0]) ]
    no_profits_week = [choose_index_values_2(dataframes,l) for l in range(len(dataframes)) if len(choose_index_values_2(dataframes,l)[0])>=1]
    # Para rellanar tabla
    num_Activos_ganancia_week = len(np.unique(np.concatenate([l[0] for l in profits_week])))
    num_Activos_perdida_week = len(np.unique(np.concatenate([l[0] for l in no_profits_week])))
    # Graficar
    semanal_positivo = subplots_positivos_negativos(profit_loss=profits_week,positivo =True, day =False)
    semanal_negativo = subplots_positivos_negativos(profit_loss=no_profits_week,positivo =False, day =False)
    
    # Realizar grafica que explique todo lo que las graficas muestran
    tabla_explicativa = pd.DataFrame(columns = ["No. de Activos \n utilizados en total","No. de Activos que \n solo producieron pérdida",
                                               "No. de Activos que \n solo producieron ganancia"],index = ["Diario","Semanal","Mensual"])
    tabla_explicativa.loc["Diario",:] = [num_Activos,num_Activos_perdida_day,num_Activos_ganancia_day]
    tabla_explicativa.loc["Semanal",:] = [num_Activos,num_Activos_perdida_week,num_Activos_ganancia_week]
    num_Activos_ganancia_month = pd.DataFrame(data.groupby(["Symbol"]).sum()["Profit"][data.groupby(["Symbol"]).sum()["Profit"]>0])
    num_Activos_perdida_month = pd.DataFrame(data.groupby(["Symbol"]).sum()["Profit"][data.groupby(["Symbol"]).sum()["Profit"]<0])
    tabla_explicativa.loc["Mensual",:] = [num_Activos,num_Activos_perdida_month.shape[0],num_Activos_ganancia_month.shape[0]]
    
    
    # Activos en lo que se recomienda invertir
    data = data_mensual
    # Se toman los activos mensuales con rendimiento positivo, se ordenan de mayor a menor y se juntan en un solo vector
    concatenados = np.concatenate(np.sort(pd.DataFrame(data.groupby(["Symbol"]).sum()["Profit"][data.groupby(["Symbol"]).sum()["Profit"]>0]).values))
    # Como queremos saber cuantos activos explican por lo menos el 95% del rendimiento 
    # quitamos aquellos que resultan insignificativos
    ultimo_significativo = (np.sort(concatenados)[::-1])[np.cumsum(np.sort(concatenados)[::-1])/np.sum(concatenados)<=.95][-1]
    activos_deseados = num_Activos_ganancia_month[num_Activos_ganancia_month.values>=ultimo_significativo].dropna()
    graficas = {"DiarioP":diario_positivo,"DiarioN":diario_negativo,"SemanalP":semanal_positivo,"SemanalN":semanal_negativo}

    return {"datos":[tabla_explicativa, activos_deseados],"grafica":graficas,"escala":1- (abs(-num_Activos+len(activos_deseados.values)))/num_Activos,
            "explicacion": """Paradox Choice: Sesgo donde se argumenta que mientras más elecciones se tiene más ansiedad se provoca, por lo tanto, disminuir nuestra posibilidad de elección puede resultar mejor. En el área de Trading (en nuestro análisis) lo que se busca en reducir el número de instrumentos con los que se opera y que explican la mayor parte de tu rendimiento."""}

In [22]:
# Probar que la función si trabaja
data = pd.read_excel("archivo_tradeview_2.xlsx")
prueba = f_bf_Paradox_Choice(data)

In [23]:
prueba.keys() # Nombre de los valores que están en el diccionario

dict_keys(['datos', 'grafica', 'escala', 'explicacion'])

In [24]:
prueba["datos"][0] # Información general sobre el comportamiento en las ganancias, pérdidas del inversionisa en distintos plazos

,No. de Activos utilizados en total,No. de Activos que solo producieron pérdida,No. de Activos que solo producieron ganancia
Diario,11,9,10
Semanal,11,7,9
Mensual,11,2,9


In [25]:
prueba["datos"][1] # Información general sobre el comportamiento en las ganancias, pérdidas del inversionisa en distintos plazos # Activos que explican más del 95% de los ingresos del inversionistas y en los que posiblemente debería enforcse más

,Profit
Symbol,
eurgbp,16.07
gbpusd,38.99
usdcad,22.43
usdmxn,36.83
xauusd,135.43


In [26]:
prueba["grafica"].keys() # Las graficas existentes

dict_keys(['DiarioP', 'DiarioN', 'SemanalP', 'SemanalN'])

In [27]:
prueba["grafica"]["DiarioP"]
fig.show()

In [28]:
prueba["grafica"]["DiarioN"]
fig.show()

In [29]:
prueba["grafica"]["SemanalP"]
fig.show()

In [30]:
prueba["grafica"]["SemanalN"]
fig.show()

In [31]:
prueba["escala"] # Del total de activos que maneja solo debería de hacer trading con el 45% de los mismos

0.4545454545454546

In [32]:
prueba["explicacion"]

'Paradox Choice: Sesgo donde se argumenta que mientras más elecciones se tiene más ansiedad se provoca, por lo tanto, disminuir nuestra posibilidad de elección puede resultar mejor. En el área de Trading (en nuestro análisis) lo que se busca en reducir el número de instrumentos con los que se opera y que explican la mayor parte de tu rendimiento.'

### <font color = navy> Conclusiones


Mediante un análisis profundo sobre el comportamiento del inversionista, pudimos demostrar que existe cierto sesgo cuando las posibilidades de elección de inversión incrementan. Hicimos un análisis donde se muestra que sin importar el número de activos en los que se invierta, jamás serán más de 5 los que realmente expliquen la mayor ganancia de tu inversión, por lo tanto, quizás sería una mejor opción prestar más atención e inversión en aquellos activos que traen la mayor ganancia a tu cuenta.

### <font color = navy> Bibliografía

http://www.sarvervrooman.wfadv.com/files/68283/18_Key_Behavioral_Sarver-Vrooman_WFA_4cF_hi-res.pdf